In [ ]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

annFile=f'/content/drive/MyDrive/instances_val2017.json'

coco=COCO(annFile)

loading annotations into memory...
Done (t=2.25s)
creating index...
index created!


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import torch
#model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
import tensorflow as tf
from torchvision.models.detection import maskrcnn_resnet50_fpn

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

model = maskrcnn_resnet50_fpn(pretrained=True, progress=False)
model = model.eval()

# model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_mobilenet_v3_large', pretrained=True)

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


In [ ]:
inst_classes = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

inst_class_to_idx = {cls: idx for (idx, cls) in enumerate(inst_classes)}

In [ ]:
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# create a color pallette, selecting a color for each class
palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
colors = (colors % 255).numpy().astype("uint8")

# get all images containing given categories, select one at random
catIds = coco.getCatIds(catNms=['person','dog','skateboard','elephant','cat']);
imgIds = coco.getImgIds(catIds=catIds );
numimages = len(imgIds)
for i in range(15):
  img = coco.loadImgs(imgIds[i])[0]
  input_image = io.imread(img['coco_url'])
  plt.imshow(input_image)
  input_tensor = preprocess(input_image)
  input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

  # move the input and model to GPU for speed if available
  if torch.cuda.is_available():
      input_batch = input_batch.to('cuda')
      model.to('cuda')

  with torch.no_grad():
      output = model(input_batch)

  dog1_output = output[0]
  dog1_masks = dog1_output['masks']

  unique_labels = np.unique(output)
  print("For the first dog, the following instances were detected:")
  print([inst_classes[label] for label in dog1_output['labels']])

  i += 1


['person', 'bowl', 'dining table', 'cup', 'cup', 'bottle', 'umbrella', 'handbag', 'person', 'car', 'sink', 'bottle', 'car']
['traffic light', 'traffic light', 'car', 'car', 'tv', 'car', 'car', 'surfboard', 'parking meter']
['car', 'car', 'car', 'traffic light', 'parking meter', 'car', 'car', 'person', 'car', 'traffic light']
['person', 'bench', 'person', 'car', 'car', 'car', 'car', 'bench', 'person', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'bench', 'parking meter', 'car', 'car', 'person', 'person', 'car']
['car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'bottle', 'traffic light', 'car']
['car', 'chair', 'toilet', 'cell phone', 'car', 'refrigerator', 'tv']
['toilet', 'car', 'clock', 'car', 'car', 'truck', 'truck', 'car', 'cell phone', 'tv', 'frisbee', 'oven', 'parking meter', 'bus', 'tv', 'motorcycle', 'traffic light', 'person', 'car']
['car', 'car', 'car', 'car', 'bottle', 'skateboard']
['car', 'tv', 'car', 'car', 'tv', 'bench', 'bus', 'car', 'tv', 'car', 'truck